In [1]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('Datasets\train.csv',engine = 'python')
df_test = pd.read_csv('Datasets\test.csv')
df_add = pd.read_csv('Datasets\addresses.csv').set_index('address')
df_ll = pd.read_csv('Datasets\latlons.csv').set_index('address')

In [16]:
df_add = df_add.join(df_ll,how = 'left')

In [19]:
df_add = df_add.set_index('ticket_id')

In [24]:
df_train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [25]:
useless_labels = ['agency_name', 'violation_street_number', 'mailing_address_str_number', 'state', 
                'disposition', 'violator_name', 'zip_code', 'country', 'city',
                'inspector_name', 'violation_street_name',
                'violation_zip_code', 'violation_description', 'mailing_address_str_name',
                'non_us_str_code',
                'ticket_issued_date', 'hearing_date', 'violation_code', 'grafitti_status']

train_leaks = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status', 
    ]

In [27]:
df_train.drop(useless_labels,inplace = True,axis = 1)
df_test.drop(useless_labels,inplace = True,axis = 1)

In [29]:
df_train = df_train[(df_train.compliance == 1.0) | (df_train.compliance == 0.0)]

In [31]:
df_train.drop(train_leaks,inplace = True,axis = 1)

c:\python\python38\lib\site-packages\pandas\core\frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

In [35]:
X_train = X_train.set_index('ticket_id')

In [61]:
X_train

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon
ticket_id,,,,,,,,,
22056,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.390729,-83.124268
27586,750.0,20.0,10.0,75.0,0.0,0.0,855.0,42.326937,-83.135118
22046,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.145257,-83.208233
18738,750.0,20.0,10.0,75.0,0.0,0.0,855.0,42.433466,-83.023493
18735,100.0,20.0,10.0,10.0,0.0,0.0,140.0,42.388641,-83.037858
...,...,...,...,...,...,...,...,...,...
285106,200.0,20.0,10.0,20.0,0.0,0.0,250.0,42.440228,-83.154829
284650,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,42.406293,-83.171850
285125,500.0,20.0,10.0,50.0,0.0,0.0,580.0,42.366529,-83.141897


In [56]:
X_train = X_train.join(df_add,how = 'left')

In [63]:
X_test = df_test

In [65]:
X_test = X_test.set_index('ticket_id')

In [69]:
X_test

array([False])

In [70]:
X_train.describe()

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon
count,159880.000000,159880.0,159880.0,159880.000000,159880.000000,159880.0,159880.000000,159878.000000,159878.000000
mean,357.035295,20.0,10.0,33.651512,0.195959,0.0,420.650218,42.389747,-83.111530
std,675.655580,0.0,0.0,67.692916,4.290344,0.0,742.555062,0.037120,0.093515
min,0.000000,20.0,10.0,0.000000,0.000000,0.0,0.000000,42.145257,-84.403766
25%,200.000000,20.0,10.0,10.000000,0.000000,0.0,250.000000,42.364536,-83.185854
50%,250.000000,20.0,10.0,25.000000,0.000000,0.0,305.000000,42.392575,-83.124521
75%,250.000000,20.0,10.0,25.000000,0.000000,0.0,305.000000,42.419160,-83.033202
max,10000.000000,20.0,10.0,1000.000000,350.000000,0.0,11030.000000,44.785389,-82.512936


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

In [85]:
param_grid = {'C':[0.01,0.1,1,10,100]}

In [90]:
X_train.lat.fillna(method='pad', inplace=True)
X_train.lon.fillna(method='pad', inplace=True)
#X_train.state.fillna(method='pad', inplace=True)

X_test.lat.fillna(method='pad', inplace=True)
X_test.lon.fillna(method='pad', inplace=True)

In [89]:
X_test = X_test.join(df_add,how = 'left')

In [95]:
grid_cv = GridSearchCV(LogisticRegression(),param_grid=param_grid,cv = 5,scoring='roc_auc')
grid_cv.fit(X_train_scaled,y_train)

c:\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='roc_auc',
             verbose=0)

In [96]:
grid_cv.best_score_

0.7547261492238596

In [93]:
grid_cv.best_params_

{'C': 10}

In [94]:
s = MinMaxScaler()
X_train_scaled = s.fit_transform(X_train)

In [97]:
X_test_sclaed = s.transform(X_test)

In [105]:
y = grid_cv.predict_proba(X_test_sclaed)[:,1]

In [106]:
y

array([0.06868135, 0.01156404, 0.07890853, ..., 0.08395849, 0.08395028,
       0.11552441])

In [100]:
result = X_test.index

In [101]:
result

Int64Index([284932, 285362, 285361, 285338, 285346, 285345, 285347, 285342,
            285530, 284989,
            ...
            376482, 376480, 376479, 376481, 376483, 376496, 376497, 376499,
            376500, 369851],
           dtype='int64', name='ticket_id', length=61001)

In [102]:
result = pd.Series??

In [107]:
result = pd.Series(data = y,index = result)

In [108]:
result

ticket_id
284932    0.068681
285362    0.011564
285361    0.078909
285338    0.077714
285346    0.095258
            ...   
376496    0.012702
376497    0.012702
376499    0.083958
376500    0.083950
369851    0.115524
Length: 61001, dtype: float64

## Coursera Assignment

In [109]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

def blight_model():
    # Your code here
    df_train = pd.read_csv('Datasets\train.csv',engine = 'python')
    df_test = pd.read_csv('Datasets\test.csv', encoding = "ISO-8859-1",engine='python')
    df_add = pd.read_csv('Datasets\addresses.csv',encoding = "ISO-8859-1",engine='python').set_index('address')
    df_ll = pd.read_csv('Datasets\latlons.csv',encoding = "ISO-8859-1",engine='python').set_index('address')
    
    
    df_add = df_add.join(df_ll,how = 'left')
    df_add = df_add.set_index('ticket_id')
    
    
    
    useless_labels = ['agency_name', 'violation_street_number', 'mailing_address_str_number', 'state', 
                'disposition', 'violator_name', 'zip_code', 'country', 'city',
                'inspector_name', 'violation_street_name',
                'violation_zip_code', 'violation_description', 'mailing_address_str_name',
                'non_us_str_code',
                'ticket_issued_date', 'hearing_date', 'violation_code', 'grafitti_status']

    train_leaks = [
        'balance_due',
        'collection_status',
        'compliance_detail',
        'payment_amount',
        'payment_date',
        'payment_status', 
    ]
    
    
    
    
    df_train.drop(useless_labels,inplace = True,axis = 1)
    df_test.drop(useless_labels,inplace = True,axis = 1)
    df_train = df_train[(df_train.compliance == 1.0) | (df_train.compliance == 0.0)]
    df_train.drop(train_leaks,inplace = True,axis = 1)
    
    
    X_train = df_train.iloc[:,:-1]
    y_train = df_train.iloc[:,-1]
    
    
    X_train = X_train.set_index('ticket_id')
    
    X_train = X_train.join(df_add,how = 'left')
    
    X_test = df_test
    X_test = X_test.set_index('ticket_id')
    X_test = X_test.join(df_add,how = 'left')
    
    X_train.lat.fillna(method='pad', inplace=True)
    X_train.lon.fillna(method='pad', inplace=True)
    X_test.lat.fillna(method='pad', inplace=True)
    X_test.lon.fillna(method='pad', inplace=True)
    
    param_grid = {'C':[0.01,0.1,1,10,100]}
    
    s = MinMaxScaler()
    X_train_scaled = s.fit_transform(X_train)
    X_test_sclaed = s.transform(X_test)
    
    grid_cv = GridSearchCV(LogisticRegression(),param_grid=param_grid,cv = 5,scoring='roc_auc')
    grid_cv.fit(X_train_scaled,y_train)
    
    y = grid_cv.predict_proba(X_test_sclaed)[:,1]
    
    result = X_test.index
    
    result = pd.Series(data = y,index = result,name = 'compliance')
    
    
    return result # Your answer here

In [110]:
blight_model()

c:\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ticket_id
284932    0.068681
285362    0.011564
285361    0.078909
285338    0.077714
285346    0.095258
            ...   
376496    0.012702
376497    0.012702
376499    0.083958
376500    0.083950
369851    0.115524
Name: compliance, Length: 61001, dtype: float64

In [117]:
from sklearn.metrics import precision_score,recall_score,precision_recall_curve
precision_recall_curve??
precision_score

<function sklearn.metrics._classification.precision_score(y_true, y_pred, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>

In [ ]:
from sklearn.scorer import Sc